In [12]:
import sys
import hashlib
from PIL import Image, UnidentifiedImageError
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes

# Função para carregar imagens no Colab
def upload_image():
    from google.colab import files
    # Faz o upload da imagem
    uploaded = files.upload()
    for filename in uploaded.keys():
        print(f"Arquivo carregado: {filename}")
        print("-"*100)
        print("-"*100)
        print(f"Use este caminho no menu: /content/{filename}")
        print("-"*100)
        print("-"*100)
    print("Nota: O arquivo foi salvo no diretório '/content/'. Copie e use o caminho exibido acima quando solicitado no menu.")

# Função para embutir texto na imagem
def embed_text_in_image(image_path, output_path, text):
    try:
        img = Image.open(image_path)
        binary_text = ''.join([format(ord(i), "08b") for i in text]) + "00000000"
        img_data = list(img.getdata())

        if len(binary_text) > len(img_data):
            raise ValueError("Texto muito longo para ser embutido nesta imagem.")

        for i in range(len(binary_text)):
            pixel = list(img_data[i])
            pixel[0] = (pixel[0] & ~1) | int(binary_text[i])
            img_data[i] = tuple(pixel)

        img.putdata(img_data)
        img.save(output_path)
        print(f"Texto embutido com sucesso em {output_path}")
    except FileNotFoundError:
        print(f"Erro: Arquivo '{image_path}' não encontrado.")
    except UnidentifiedImageError:
        print(f"Erro: O arquivo '{image_path}' não é uma imagem válida.")
    except ValueError as e:
        print(f"Erro: {e}")
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")

# Função para recuperar texto embutido na imagem
def extract_text_from_image(image_path):
    try:
        img = Image.open(image_path)
        img_data = list(img.getdata())

        binary_text = ""
        for pixel in img_data:
            binary_text += str(pixel[0] & 1)

        text = ''.join([chr(int(binary_text[i:i+8], 2)) for i in range(0, len(binary_text), 8)])
        return text.split("\x00")[0]
    except FileNotFoundError:
        print(f"Erro: Arquivo '{image_path}' não encontrado.")
        return ""
    except UnidentifiedImageError:
        print(f"Erro: O arquivo '{image_path}' não é uma imagem válida.")
        return ""
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")
        return ""

# Função para calcular o hash da imagem
def calculate_image_hash(image_path):
    try:
        with open(image_path, "rb") as file:
            img_bytes = file.read()
            return hashlib.sha256(img_bytes).hexdigest()
    except FileNotFoundError:
        print(f"Erro: Arquivo '{image_path}' não encontrado.")
        return None
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")
        return None

# Funções para encriptação e decriptação usando chave pública e privada
def generate_keys():
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    public_key = private_key.public_key()
    return private_key, public_key

def encrypt_message(public_key, message):
    try:
        ciphertext = public_key.encrypt(
            message.encode(),
            padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
        )
        return ciphertext
    except Exception as e:
        print(f"Erro ao encriptar a mensagem: {e}")
        return None

def decrypt_message(private_key, ciphertext):
    try:
        plaintext = private_key.decrypt(
            ciphertext,
            padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
        )
        return plaintext.decode()
    except ValueError:
        print("Erro: O texto fornecido não é válido para decriptação.")
    except TypeError:
        print("Erro: O texto fornecido deve estar no formato de bytes.")
    except Exception as e:
        print(f"Erro ao decriptar a mensagem: {e}")
        return None

# Menu principal
def main_menu():
    private_key, public_key = generate_keys()
    while True:
        print("\nMenu de Opções:")
        print("(0) Carregar imagem no Colab")
        print("(1) Embutir texto em imagem")
        print("(2) Recuperar texto de imagem")
        print("(3) Gerar hash de imagens")
        print("(4) Encriptar mensagem")
        print("(5) Decriptar mensagem")
        print("(S ou s) Sair")

        choice = input("Escolha uma opção: ").strip().lower()

        if choice == "0":
            upload_image()

        elif choice == "1":
            image_path = input("Caminho da imagem original: ").strip()
            output_path = input("Caminho para salvar a imagem modificada (inclua nome e extensão, ex.: /content/imagem_modificada.png): ").strip()
            text = input("Texto a embutir: ").strip()
            embed_text_in_image(image_path, output_path, text)

        elif choice == "2":
            image_path = input("Caminho da imagem modificada: ").strip()
            text = extract_text_from_image(image_path)
            if text:
                print(f"Texto recuperado: {text}")
            else:
                print("Nenhum texto foi recuperado da imagem.")

        elif choice == "3":
            image_path_original = input("Caminho da imagem original: ").strip()
            image_path_modified = input("Caminho da imagem modificada: ").strip()
            hash_original = calculate_image_hash(image_path_original)
            hash_modified = calculate_image_hash(image_path_modified)
            if hash_original and hash_modified:
                print(f"Hash da imagem original: {hash_original}")
                print(f"Hash da imagem modificada: {hash_modified}")
            else:
                print("Erro ao calcular os hashes das imagens.")

        elif choice == "4":
            message = input("Mensagem a encriptar: ").strip()
            ciphertext = encrypt_message(public_key, message)
            if ciphertext:
                print(f"Mensagem encriptada com sucesso: {ciphertext}")
            else:
                print("Erro ao encriptar a mensagem.")

        elif choice == "5":
            try:
                ciphertext = eval(input("Insira a mensagem encriptada (em bytes): "))
                plaintext = decrypt_message(private_key, ciphertext)
                if plaintext:
                    print(f"Mensagem decriptada: {plaintext}")
            except SyntaxError:
                print("Erro: O texto inserido deve estar no formato correto de bytes (ex.: b'mensagem').")

        elif choice == "s":
            print("Encerrando a aplicação...")
            break
        else:
            print("Opção inválida. Tente novamente.")

# Inicia o programa
if __name__ == "__main__":
    main_menu()



Menu de Opções:
(0) Carregar imagem no Colab
(1) Embutir texto em imagem
(2) Recuperar texto de imagem
(3) Gerar hash de imagens
(4) Encriptar mensagem
(5) Decriptar mensagem
(S ou s) Sair
Escolha uma opção: 0


Saving caracteristicas-cavalos-saudaveis-artigos-cursos-cpt.jpg to caracteristicas-cavalos-saudaveis-artigos-cursos-cpt.jpg
Arquivo carregado: caracteristicas-cavalos-saudaveis-artigos-cursos-cpt.jpg
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Use este caminho no menu: /content/caracteristicas-cavalos-saudaveis-artigos-cursos-cpt.jpg
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Nota: O arquivo foi salvo no diretório '/content/'. Copie e use o caminho exibido acima quando solicitado no menu.

Menu de Opções:
(0) Carregar imagem no Colab
(1) Embutir texto em imagem
(2) Recuperar texto de imagem
(3) Gerar hash de imagens
(4) Encriptar mensagem
(5) Decriptar mensagem
(S ou s) Sair